In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
# Cargar el dataset de IMDB (clasificación de sentimientos)
dataset = load_dataset('imdb')

##### Tokenizamos el texto: BERT requiere que el texto sea tokenizado de manera específica, por lo que utilizaremos el tokenizador de BERT para convertir las frases en secuencias de tokens.

In [ ]:
# Cargar el tokenizador BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Función para tokenizar los datos
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True)

# Aplicar el tokenizador a todo el dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Dividir el dataset en entrenamiento y evaluación
train_dataset = tokenized_datasets['train']
test_dataset = tokenized_datasets['test']

In [ ]:
# Cargar el modelo BERT preentrenado para clasificación
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

##### Configuramos los parámetros de entrenamiento: Vamos a configurar los parámetros de entrenamiento utilizando TrainingArguments para el Trainer de Hugging Face.

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # directorio donde se guardan los resultados
    num_train_epochs=3,              # número de épocas de entrenamiento
    per_device_train_batch_size=8,   # tamaño del batch para el entrenamiento
    per_device_eval_batch_size=8,    # tamaño del batch para la evaluación
    warmup_steps=500,                # número de pasos de warmup
    weight_decay=0.01,               # regularización L2
    logging_dir='./logs',            # directorio de logs
    logging_steps=10,
    evaluation_strategy="epoch",     # Evaluar al final de cada época
)

##### Definimos las métricas de evaluación: Vamos a definir una función para evaluar el modelo en base a la precisión (accuracy) para ver cómo está funcionando durante el entrenamiento.

In [ ]:
def compute_metrics(p):
    predictions, labels = p
    predictions = predictions.argmax(axis=-1)
    return accuracy_score(labels, predictions)

##### Creamos el objeto Trainer: Usamos el Trainer de Hugging Face, que es una forma muy fácil de entrenar el modelo con un conjunto de datos.

In [ ]:
trainer = Trainer(
    model=model,                         # El modelo a entrenar
    args=training_args,                  # Los parámetros de entrenamiento
    train_dataset=train_dataset,         # Dataset de entrenamiento
    eval_dataset=test_dataset,           # Dataset de evaluación
    compute_metrics=compute_metrics      # Función para computar las métricas
)

In [ ]:
# Entrenamos el modelo !!!
trainer.train()

In [ ]:
# Evaluamos el modelo en el conjunto de test
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

In [ ]:
# Guardamos el modelo entrenado
model.save_pretrained('./bert_finetuned')
tokenizer.save_pretrained('./bert_finetuned')
